# Assignment #7 Kaggle Full Data Set Prep

In [2]:
#Load in necessary libraries
import pandas as pd  # data frame operations  
import numpy as np  # arrays and math functions
import matplotlib.pyplot as plt  # static plotting
import re # regular expressions
import scipy
import os # Operation System
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import datetime
import cv2

In [4]:
#Establish Current Working Directory
!pwd

/Users/mikesoukup/Desktop/NU MSDS/MSDS 422/Module 7


In [6]:
#Start creating directory paths:
work_dir = "/Users/mikesoukup/Desktop/NU MSDS/MSDS 422/Module 7"
data_dir = work_dir + "/kaggledata"
print(data_dir)

/Users/mikesoukup/Desktop/NU MSDS/MSDS 422/Module 7/kaggledata


In [10]:
#Create path do the directories where the original raster files are located
train_dir_name = data_dir + "/train"
test_dir_name = data_dir + "/test1"

#Standard Functions
#Sorting of file names facilitated by
def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

#Generate list of file names

def directory_list (dir_name, str1):
    start_list = os.listdir(dir_name)
    end_list = []
    for file in start_list:
        if (file.startswith(str1)):
            end_list.append(file)
    end_list.sort(key = alphanum_key)
    return(end_list)

In [12]:
#Retrieve Dog, Cat, and Test data:
cat_file_names = directory_list(train_dir_name,"cat")
dog_file_names = directory_list(train_dir_name,"dog")
test_file_names = directory_list(test_dir_name, "")

In [13]:
#Verify this retrieval was successful:
print("Length of Cat Files from Kaggle:",len(cat_file_names))
print("Length of Dog Files from Kaggle:",len(dog_file_names))
print("Length of Test Files from Kaggle:",len(test_file_names))

Length of Cat Files from Kaggle: 12500
Length of Dog Files from Kaggle: 12500
Length of Test Files from Kaggle: 12500


So this looks like it all checks out. This will give us 25,000 training instances and then 12,500 test images for this competition. This matches the Kaggle description of the data set.

In [16]:
#Now convert the images to a numpy array:
def parse_grayscale(image_file_path):
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)
    return(image)

def parse_color(image_file_path):
    image = cv2.imread(image_file_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return(image)
def parse_grayscale_and_resize(image_file_path, size = (64, 64)):
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, size)
    return(image)

def parse_color_and_resize(image_file_path, size = (64, 64)):
    image = cv2.imread(image_file_path, cv2.IMREAD_COLOR)
    # Default cv2 is BGR... need RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, size)
    return(image)  
    
def show_grayscale_image(image):
    plt.imshow(image, cmap = 'gray') 
    plt.axis('off')
    plt.show()

def show_color_image(image):
    plt.imshow(image) 
    plt.axis('off')
    plt.show() 

In [18]:
#Examine the dimensions of original raster images:
cats_shapes = []
for ifile in range(len(cat_file_names)):
    image_file_path = os.path.join(train_dir_name, cat_file_names[ifile])
    image = parse_color(image_file_path)
    cats_shapes.append(image.shape)
print('\n\nCat iamge file shapes:\n')
print(cats_shapes)



Cat iamge file shapes:

[(374, 500, 3), (280, 300, 3), (396, 312, 3), (414, 500, 3), (375, 499, 3), (144, 175, 3), (303, 400, 3), (499, 495, 3), (345, 461, 3), (425, 320, 3), (499, 489, 3), (410, 431, 3), (224, 300, 3), (315, 499, 3), (267, 320, 3), (353, 405, 3), (258, 448, 3), (375, 499, 3), (374, 500, 3), (223, 320, 3), (374, 500, 3), (499, 431, 3), (345, 500, 3), (256, 334, 3), (374, 500, 3), (500, 345, 3), (374, 500, 3), (479, 370, 3), (270, 286, 3), (375, 499, 3), (262, 349, 3), (374, 500, 3), (374, 500, 3), (375, 499, 3), (499, 375, 3), (426, 499, 3), (311, 500, 3), (337, 499, 3), (337, 350, 3), (500, 374, 3), (383, 499, 3), (499, 333, 3), (173, 237, 3), (500, 356, 3), (102, 107, 3), (370, 500, 3), (433, 400, 3), (214, 258, 3), (93, 139, 3), (129, 180, 3), (196, 299, 3), (473, 256, 3), (300, 399, 3), (457, 492, 3), (499, 500, 3), (500, 344, 3), (332, 499, 3), (291, 335, 3), (375, 499, 3), (331, 464, 3), (176, 180, 3), (226, 328, 3), (377, 500, 3), (374, 500, 3), (374, 500, 3),

In [19]:
dogs_shapes = []
for ifile in range(len(dog_file_names)):
    image_file_path = os.path.join(train_dir_name, dog_file_names[ifile])
    image = parse_color(image_file_path)
    dogs_shapes.append(image.shape)    
print('\n\nDog image file shapes:\n') 
print(dogs_shapes)



Dog image file shapes:

[(375, 499, 3), (499, 327, 3), (199, 187, 3), (375, 499, 3), (287, 300, 3), (376, 499, 3), (488, 499, 3), (264, 299, 3), (500, 469, 3), (500, 368, 3), (292, 269, 3), (101, 135, 3), (161, 98, 3), (428, 362, 3), (386, 500, 3), (374, 500, 3), (380, 500, 3), (335, 272, 3), (348, 215, 3), (225, 299, 3), (332, 500, 3), (499, 415, 3), (371, 499, 3), (403, 499, 3), (500, 274, 3), (375, 499, 3), (224, 300, 3), (375, 499, 3), (432, 287, 3), (376, 500, 3), (457, 466, 3), (500, 297, 3), (375, 499, 3), (375, 499, 3), (374, 500, 3), (243, 323, 3), (240, 319, 3), (371, 400, 3), (374, 499, 3), (356, 324, 3), (197, 149, 3), (240, 319, 3), (374, 500, 3), (249, 375, 3), (375, 499, 3), (320, 425, 3), (299, 243, 3), (477, 500, 3), (360, 480, 3), (375, 499, 3), (199, 157, 3), (500, 433, 3), (377, 499, 3), (267, 396, 3), (284, 400, 3), (420, 500, 3), (499, 492, 3), (400, 302, 3), (281, 499, 3), (331, 500, 3), (304, 448, 3), (342, 499, 3), (449, 337, 3), (499, 401, 3), (500, 419, 3),

In [21]:
test_shapes = []
for ifile in range(len(test_file_names)):
    image_file_path = os.path.join(test_dir_name, test_file_names[ifile])
    image = parse_color(image_file_path)
    test_shapes.append(image.shape)    
print('\n\nTest image file shapes:\n') 
print(test_shapes)



Test image file shapes:

[(499, 381, 3), (299, 296, 3), (299, 300, 3), (288, 374, 3), (400, 399, 3), (375, 499, 3), (148, 192, 3), (234, 325, 3), (380, 480, 3), (374, 500, 3), (370, 278, 3), (465, 315, 3), (376, 500, 3), (293, 500, 3), (499, 377, 3), (374, 500, 3), (399, 228, 3), (459, 499, 3), (272, 328, 3), (232, 356, 3), (450, 443, 3), (255, 384, 3), (252, 377, 3), (500, 393, 3), (294, 499, 3), (265, 480, 3), (375, 499, 3), (500, 499, 3), (365, 500, 3), (500, 340, 3), (364, 224, 3), (288, 383, 3), (380, 499, 3), (375, 499, 3), (278, 149, 3), (375, 499, 3), (387, 500, 3), (119, 150, 3), (375, 499, 3), (312, 499, 3), (375, 499, 3), (400, 369, 3), (374, 500, 3), (240, 227, 3), (400, 399, 3), (50, 41, 3), (300, 399, 3), (450, 342, 3), (500, 399, 3), (308, 368, 3), (375, 499, 3), (374, 500, 3), (499, 438, 3), (321, 500, 3), (348, 500, 3), (369, 416, 3), (197, 189, 3), (349, 350, 3), (199, 190, 3), (336, 447, 3), (374, 500, 3), (239, 320, 3), (303, 200, 3), (374, 500, 3), (357, 500, 3),

In [22]:
#S6 Create Numpy Gray Scale Arrays 64 x 64
#----------------------------------------------------------------------
print('\nProcessing image files to 64x64 gryscale arrays')
# Create cats_1000_64_64_1 and numpy array for 1000 cat images in grayscale
cats_12500_64_64_1 = np.zeros((12500, 64, 64,1))  
for ifile in range(len(cat_file_names)):
    image_file_path = os.path.join(train_dir_name, cat_file_names[ifile])
    image = parse_grayscale_and_resize(image_file_path, size = (64, 64))
    cats_12500_64_64_1[ifile,:,:,0] = image
       
# Create dogs_1000_64_64_1 and numpy array for 1000 dog images in grayscale   
dogs_12500_64_64_1 = np.zeros((12500, 64, 64,1))  
for ifile in range(len(dog_file_names)):
    image_file_path = os.path.join(train_dir_name, dog_file_names[ifile])
    image = parse_grayscale_and_resize(image_file_path, size = (64, 64))
    dogs_12500_64_64_1[ifile,:,:,0] = image
    
# Create dogs_1000_64_64_1 and numpy array for 1000 dog images in grayscale   
test_12500_64_64_1 = np.zeros((12500, 64, 64,1))  
for ifile in range(len(test_file_names)):
    image_file_path = os.path.join(test_dir_name, test_file_names[ifile])
    image = parse_grayscale_and_resize(image_file_path, size = (64, 64))
    test_12500_64_64_1[ifile,:,:,0] = image


Processing image files to 64x64 gryscale arrays


In [23]:
#S6 Create Numpy Gray Scale Arrays 128 x 128
#----------------------------------------------------------------------
print('\nProcessing image files to 128x128 gryscale arrays')
# Create cats_1000_64_64_1 and numpy array for 1000 cat images in grayscale
cats_12500_128_128_1 = np.zeros((12500, 128, 128,1))  
for ifile in range(len(cat_file_names)):
    image_file_path = os.path.join(train_dir_name, cat_file_names[ifile])
    image = parse_grayscale_and_resize(image_file_path, size = (128, 128))
    cats_12500_128_128_1[ifile,:,:,0] = image
       
# Create dogs_1000_64_64_1 and numpy array for 1000 dog images in grayscale   
dogs_12500_128_128_1 = np.zeros((12500, 128, 128,1))  
for ifile in range(len(dog_file_names)):
    image_file_path = os.path.join(train_dir_name, dog_file_names[ifile])
    image = parse_grayscale_and_resize(image_file_path, size = (128, 128))
    dogs_12500_128_128_1[ifile,:,:,0] = image
    
# Create dogs_1000_64_64_1 and numpy array for 1000 dog images in grayscale   
test_12500_128_128_1 = np.zeros((12500, 128, 128,1))  
for ifile in range(len(test_file_names)):
    image_file_path = os.path.join(test_dir_name, test_file_names[ifile])
    image = parse_grayscale_and_resize(image_file_path, size = (128, 128))
    test_12500_128_128_1[ifile,:,:,0] = image


Processing image files to 128x128 gryscale arrays


In [26]:
#S6 Create Numpy Color Arrays 64 x 64
#----------------------------------------------------------------------
print('\nProcessing image files to 64x64 color arrays')
# Create cats_1000_64_64_1 and numpy array for 1000 cat images in grayscale
cats_12500_64_64_3 = np.zeros((12500, 64, 64,3))  
for ifile in range(len(cat_file_names)):
    image_file_path = os.path.join(train_dir_name, cat_file_names[ifile])
    image = parse_color_and_resize(image_file_path, size = (64, 64))
    cats_12500_64_64_3[ifile,:,:,:] = image
       
# Create dogs_1000_64_64_1 and numpy array for 1000 dog images in grayscale   
dogs_12500_64_64_3 = np.zeros((12500, 64, 64,3))  
for ifile in range(len(dog_file_names)):
    image_file_path = os.path.join(train_dir_name, dog_file_names[ifile])
    image = parse_color_and_resize(image_file_path, size = (64, 64))
    dogs_12500_64_64_3[ifile,:,:,:] = image
    
# Create dogs_1000_64_64_1 and numpy array for 1000 dog images in grayscale   
test_12500_64_64_3 = np.zeros((12500, 64, 64,3))  
for ifile in range(len(test_file_names)):
    image_file_path = os.path.join(test_dir_name, test_file_names[ifile])
    image = parse_color_and_resize(image_file_path, size = (64, 64))
    test_12500_64_64_3[ifile,:,:,:] = image


Processing image files to 64x64 color arrays


In [27]:
#S6 Create Numpy Color Arrays 128 x 128
#----------------------------------------------------------------------
print('\nProcessing image files to 128x128 color arrays')
# Create cats_1000_64_64_1 and numpy array for 1000 cat images in grayscale
cats_12500_128_128_3 = np.zeros((12500, 128, 128,3))  
for ifile in range(len(cat_file_names)):
    image_file_path = os.path.join(train_dir_name, cat_file_names[ifile])
    image = parse_color_and_resize(image_file_path, size = (128, 128))
    cats_12500_128_128_3[ifile,:,:,:] = image
       
# Create dogs_1000_64_64_1 and numpy array for 1000 dog images in grayscale   
dogs_12500_128_128_3 = np.zeros((12500, 128, 128,3))  
for ifile in range(len(dog_file_names)):
    image_file_path = os.path.join(train_dir_name, dog_file_names[ifile])
    image = parse_color_and_resize(image_file_path, size = (128, 128))
    dogs_12500_128_128_3[ifile,:,:,:] = image
    
# Create dogs_1000_64_64_1 and numpy array for 1000 dog images in grayscale   
test_12500_128_128_3 = np.zeros((12500, 128, 128,3))  
for ifile in range(len(test_file_names)):
    image_file_path = os.path.join(test_dir_name, test_file_names[ifile])
    image = parse_color_and_resize(image_file_path, size = (128, 128))
    test_12500_128_128_3[ifile,:,:,:] = image


Processing image files to 128x128 color arrays


In [28]:
outdir = work_dir + "/raster_npy_files"
os.mkdir(outdir)

np.save(os.path.join(outdir, 'cats_12500_64_64_1'),cats_12500_64_64_1)
np.save(os.path.join(outdir, 'dogs_12500_64_64_1'),dogs_12500_64_64_1)
np.save(os.path.join(outdir, 'test_12500_64_64_1'),test_12500_64_64_1)

np.save(os.path.join(outdir, 'cats_12500_128_128_1'),cats_12500_128_128_1)
np.save(os.path.join(outdir, 'dogs_12500_128_128_1'),dogs_12500_128_128_1)
np.save(os.path.join(outdir, 'test_12500_128_128_1'),test_12500_128_128_1)

np.save(os.path.join(outdir, 'cats_12500_64_64_3'),cats_12500_64_64_3)
np.save(os.path.join(outdir, 'dogs_12500_64_64_3'),dogs_12500_64_64_3)
np.save(os.path.join(outdir, 'test_12500_64_64_3'),test_12500_64_64_3)

np.save(os.path.join(outdir, 'cats_12500_128_128_3'),cats_12500_128_128_3)
np.save(os.path.join(outdir, 'dogs_12500_128_128_3'),dogs_12500_128_128_3)
np.save(os.path.join(outdir, 'test_12500_128_128_3'),test_12500_128_128_3)

In [30]:
plt.imshow(cats_12500_64_64_1[1], cmap = 'gray')
plt.show()